In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import pyodbc
from datetime import datetime
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import as_strided
%matplotlib inline
import gc
import scipy
from scipy import stats
import os

In [ ]:
def fill_na_until_last_valid(x):
    result = x.copy()
    result.loc[:result.last_valid_index()] = result.loc[:result.last_valid_index()].fillna(method='ffill'
                                                                                          )
    return result    

In [ ]:
def make_rebal_key(date):
    year  = str(date)[:4]
    month = str(date)[4:6]
    
    if month =='01':
        return str(int(year)-1)+'12'
    else:
        if len(str(int(month)-1)) == 1:
            return str(int(year))+'0'+str(int(month)-1)
        else:
            return str(int(year))+str(int(month)-1)

In [ ]:
def get_min_corr_code(df_corr,code_cols):
    temp = df_corr[code_cols]
    return temp.min(axis=1).sort_values().head(1).index[0]

# 匯入國內及海外基金基本資料數據 ( 國內上市 國內清算 國外上市 )

In [ ]:
df_domestic_fund_info = pd.read_csv('./fund_data/fund_domestic_info_listed.txt',engine='python')
df_domestic_fund_info = df_domestic_fund_info[[ 'TW' not in x  for x in df_domestic_fund_info['代號'] ]]
df_domestic_fund_info = df_domestic_fund_info.drop_duplicates(subset='代號',keep='last')
df_domestic_fund_info.info()

In [ ]:
df_domestic_fund_info_delisted = pd.read_csv('./fund_data/fund_domestic_info_delisted.txt',engine='python')
df_domestic_fund_info_delisted=df_domestic_fund_info_delisted.drop_duplicates(subset='代號',keep='last')
df_domestic_fund_info_delisted.info()

In [ ]:
df_foreign_fund_info = pd.read_csv('./fund_data/fund_foreign_info_listed.txt',engine='python')
df_foreign_fund_info=df_foreign_fund_info.drop_duplicates(subset='代號',keep='last')
df_foreign_fund_info.info()

# 選擇需要欄位

In [ ]:
df_domestic_fund_info = df_domestic_fund_info[['年度', '代號', '名稱','公會分類代碼','公會分類','投信代碼', '投信名稱']]
df_domestic_fund_info_delisted = df_domestic_fund_info_delisted[['年度', '代號', '名稱','公會分類代碼','公會分類','投信代碼', '投信名稱']]
df_foreign_fund_info  = df_foreign_fund_info[['年度', '代號', '名稱','基金分類代號','基金分類名稱','總代理公司代號', '總代理公司名稱']]
df_foreign_fund_info.columns = ['年度', '代號', '名稱','公會分類代碼','公會分類','投信代碼', '投信名稱']
df_fund_info = pd.concat([df_domestic_fund_info,df_domestic_fund_info_delisted,df_foreign_fund_info])
del df_domestic_fund_info,df_domestic_fund_info_delisted,df_foreign_fund_info

# 建立組別清單，以及產出各組基金代號名單group_fund_dict

In [ ]:
group_dict = {}
group_dict['TTW50T Index']   =  [
'國內股票–一般股票型',
'國內股票–上櫃股票型',
'國內股票–中小型',
'國內股票–中概股型',
'國內股票–價值型',
'國內股票–國外募集',
'國內股票–特殊類',
'國內股票–科技類'
]
group_dict['IBOXIG Index']   =[
'海外債券投資等級型–中國大陸',
'海外債券投資等級型–亞洲新興市場',
'海外債券投資等級型–全球型',
'海外債券投資等級型–全球新興市場',
'海外債券投資等級型–其它',
'海外債券投資等級型–目標到期',
'海外債券投資等級型–短期型',
'海外債券投資等級型–美國',

'固定收益型–投資級公司債–全球',
'固定收益型–投資級公司債–全球已開發市場',
'固定收益型–投資級公司債–全球新興市場',
'固定收益型–投資級公司債–已開發歐洲',
'固定收益型–投資級公司債–美國',
'固定收益型–投資級公司債–英國'
]


group_dict['LT11TRUU Index'] = [
'固定收益型–政府債–全球',
'固定收益型–政府債–全球已開發市場',
'固定收益型–政府債–北美',
'固定收益型–政府債–已開發歐洲',
'固定收益型–政府債–美國',
'固定收益型–政府債–英國']

def make_group(x,group_dict):
    for key in group_dict.keys():
        if x in group_dict[key]:
            result = key
            break
        else:
            result = np.NaN
    return result

df_fund_info['基金分組'] = [ make_group(x,group_dict) for x in df_fund_info['公會分類']]
df_fund_info            = df_fund_info.dropna()

group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])

# 匯入基金nav資料，與指數、無風險利率資料合併

In [ ]:
index_cols = [ 'TTW50T Index', 'IBOXIG Index', 'IBOXHY Index', 'LT11TRUU Index',
       'TWSEDVD Index', 'RMSG Index' ]
rf_col     = ['GVTW10YR Index']
df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
df_index.columns=df_index.columns[1:].insert(0,'date')
df_index = df_index.set_index('date')
df_index.index =[ int(datetime.strftime(x,'%Y%m%d')) for x in df_index.index ]
df_index = df_index.fillna(method='ffill')
df_index = df_index[index_cols]
#包含下市基金
df_domestic_fund=pd.read_csv('./fund_data/fund_domestic_d.txt',encoding='CP950')
df_domestic_fund['代號'] = df_domestic_fund['代號'].astype('str')
df_domestic_fund=df_domestic_fund[[ 'TW' not in x  for x in df_domestic_fund['代號'] ]]

#包含下市基金
df_foreign_fund=pd.read_csv('./fund_data/fund_foreign_d.txt',encoding='CP950')
df_foreign_fund['代號'] = df_foreign_fund['代號'].astype('str')
df_foreign_fund=df_foreign_fund[[ 'TW' not in x  for x in df_foreign_fund['代號'] ]]

#加入指標指數

#假日不會有值
#可能會有補結的狀況 比如 20200925 下午結算早上台股部位  20200925 台灣晚上美股開盤 20200926 結算美股部位 20200926是休市日

df_domestic_fund_nav   = df_domestic_fund.pivot(index='日期',columns='代號',values='淨值').apply(lambda x : fill_na_until_last_valid(x))
df_foreign_fund_nav    = df_foreign_fund.pivot(index='日期',columns='代號',values='淨值').apply(lambda x : fill_na_until_last_valid(x))
df_fund_nav            = df_domestic_fund_nav.copy()
df_fund_nav[df_foreign_fund_nav.columns] = df_foreign_fund_nav 
df_fund_nav[df_index.columns]                  = df_index
df_fund_nav = df_fund_nav.apply(lambda x : fill_na_until_last_valid(x))

df_fund_return= df_fund_nav/df_fund_nav.shift(1) - 1

df_fund_nav['C0010176'][df_fund_nav['C0010176'].index[df_fund_return['C0010176'].argmax()]] = 16.53
df_fund_nav['98637440']=df_fund_nav['98637440'][df_fund_nav['98637440'].index >=20140903]

df_fund_return= df_fund_nav/df_fund_nav.shift(1) - 1

del df_domestic_fund,df_foreign_fund
df_fund_return = df_fund_return.loc['19950101':]
df_fund_nav    = df_fund_nav.loc['19950101':]

# 計算因子

In [ ]:
class performance_factor(object):
    def __init__(self,fund_return,fund_list,rf_col,category_average_col,benchmark_col,time_param):
        self.fund_return = fund_return
        self.fund_list   = fund_list
        self.rf_col      = rf_col
        self.category_average_col = category_average_col
        self.benchmark_col        = benchmark_col
        self.time_param           = time_param
    def aer(self):
        df_aer   = pd.DataFrame()
        aer_zero = self.fund_return[self.fund_list].subtract(0, axis=0)
        aer_zero = aer_zero.rolling(self.time_param,self.time_param).mean()

        aer_rf   = self.fund_return[self.fund_list].subtract(self.fund_return[self.rf_col].values, axis=0)
        aer_rf   = aer_rf.rolling(self.time_param,self.time_param).mean()

        aer_c    = self.fund_return[self.fund_list].subtract(self.fund_return[self.category_average_col].values, axis=0)
        aer_c    = aer_c.rolling(self.time_param,self.time_param).mean()

        aer_b    = self.fund_return[self.fund_list].subtract(self.fund_return[self.benchmark_col].values, axis=0)
        aer_b    = aer_b.rolling(self.time_param,self.time_param).mean()  
        
        aer_zero_melt = aer_zero.reset_index().melt(id_vars='日期',var_name='代號',value_name='aer_zero_'+str(self.time_param))
        aer_rf_melt = aer_rf.reset_index().melt(id_vars='日期',var_name='代號',value_name='aer_rf_'+str(self.time_param))
        aer_c_melt = aer_c.reset_index().melt(id_vars='日期',var_name='代號',value_name='aer_c_'+str(self.time_param))
        aer_b_melt = aer_b.reset_index().melt(id_vars='日期',var_name='代號',value_name='aer_b_'+str(self.time_param))
        df_aer[aer_zero_melt.columns] = aer_zero_melt
        df_aer[aer_rf_melt.columns] = aer_rf_melt
        df_aer[aer_c_melt.columns] = aer_c_melt
        df_aer[aer_b_melt.columns] = aer_b_melt
        return df_aer
    def aer_succ(self):
        df_aer_succ   = pd.DataFrame()
        aer_succ_zero = self.fund_return[self.fund_list].subtract(0, axis=0)
        aer_succ_zero = (aer_succ_zero.rolling(self.time_param,self.time_param).mean().rank(axis=1,ascending=False)<=20)*1

        aer_succ_rf   = self.fund_return[self.fund_list].subtract(self.fund_return[self.rf_col].values, axis=0)
        aer_succ_rf   = (aer_succ_rf.rolling(self.time_param,self.time_param).mean().rank(axis=1,ascending=False)<=20)*1

        aer_succ_c    = self.fund_return[self.fund_list].subtract(self.fund_return[self.category_average_col].values, axis=0)
        aer_succ_c    = (aer_succ_c.rolling(self.time_param,self.time_param).mean().rank(axis=1,ascending=False)<=20)*1

        aer_succ_b    = self.fund_return[self.fund_list].subtract(self.fund_return[self.benchmark_col].values, axis=0)
        aer_succ_b    = (aer_succ_b.rolling(self.time_param,self.time_param).mean().rank(axis=1,ascending=False)<=20)*1
        
        aer_succ_zero_melt = aer_succ_zero.reset_index().melt(id_vars='日期',var_name='代號',value_name='aer_succ_zero_'+str(self.time_param))
        aer_succ_rf_melt = aer_succ_rf.reset_index().melt(id_vars='日期',var_name='代號',value_name='aer_succ_rf_'+str(self.time_param))
        aer_succ_c_melt = aer_succ_c.reset_index().melt(id_vars='日期',var_name='代號',value_name='aer_succ_c_'+str(self.time_param))
        aer_succ_b_melt = aer_succ_b.reset_index().melt(id_vars='日期',var_name='代號',value_name='aer_succ_b_'+str(self.time_param))
        df_aer_succ[aer_succ_zero_melt.columns] = aer_succ_zero_melt
        df_aer_succ[aer_succ_rf_melt.columns] = aer_succ_rf_melt
        df_aer_succ[aer_succ_c_melt.columns] = aer_succ_c_melt
        df_aer_succ[aer_succ_b_melt.columns] = aer_succ_b_melt
        return df_aer_succ
    def mrar(self):
        df_mrar   = pd.DataFrame()
        mrar_zero = (self.fund_return[self.fund_list]+1).div(1, axis=0) ** -2
        mrar_zero = mrar_zero.rolling(self.time_param,self.time_param).mean() ** (-12/2)

        mrar_rf   = (self.fund_return[self.fund_list]+1).div(self.fund_return[self.rf_col].values+1, axis=0) ** -2
        mrar_rf   = mrar_rf.rolling(self.time_param,self.time_param).mean() ** (-12/2)

        mrar_c    = (self.fund_return[self.fund_list]+1).div(self.fund_return[self.category_average_col].values+1, axis=0) ** -2
        mrar_c    = mrar_c.rolling(self.time_param,self.time_param).mean() ** (-12/2)

        mrar_b    = (self.fund_return[self.fund_list]+1).div(self.fund_return[self.benchmark_col].values+1, axis=0) ** -2
        mrar_b    = mrar_b.rolling(self.time_param,self.time_param).mean() ** (-12/2)
        
        mrar_zero_melt = mrar_zero.reset_index().melt(id_vars='日期',var_name='代號',value_name='mrar_zero_'+str(self.time_param))
        mrar_rf_melt = mrar_rf.reset_index().melt(id_vars='日期',var_name='代號',value_name='mrar_rf_'+str(self.time_param))
        mrar_c_melt = mrar_c.reset_index().melt(id_vars='日期',var_name='代號',value_name='mrar_c_'+str(self.time_param))
        mrar_b_melt = mrar_b.reset_index().melt(id_vars='日期',var_name='代號',value_name='mrar_b_'+str(self.time_param))
        df_mrar[mrar_zero_melt.columns] = mrar_zero_melt
        df_mrar[mrar_rf_melt.columns] = mrar_rf_melt
        df_mrar[mrar_c_melt.columns] = mrar_c_melt
        df_mrar[mrar_b_melt.columns] = mrar_b_melt
        return df_mrar
    def mrar_succ(self):
        df_mrar_succ   = pd.DataFrame()
        mrar_succ_zero = (self.fund_return[self.fund_list]+1).div(1, axis=0) ** -2
        mrar_succ_zero = ((mrar_succ_zero.rolling(self.time_param,self.time_param).mean() ** (-12/2)).rank(axis=1,ascending=False)<=20)*1

        mrar_succ_rf   = (self.fund_return[self.fund_list]+1).div(self.fund_return[self.rf_col].values+1, axis=0) ** -2
        mrar_succ_rf   = ((mrar_succ_rf.rolling(self.time_param,self.time_param).mean() ** (-12/2)).rank(axis=1,ascending=False)<=20)*1

        mrar_succ_c    = (self.fund_return[self.fund_list]+1).div(self.fund_return[self.category_average_col].values+1, axis=0) ** -2
        mrar_succ_c    = ((mrar_succ_c.rolling(self.time_param,self.time_param).mean() ** (-12/2)).rank(axis=1,ascending=False)<=20)*1

        mrar_succ_b    = (self.fund_return[self.fund_list]+1).div(self.fund_return[self.benchmark_col].values+1, axis=0) ** -2
        mrar_succ_b    = ((mrar_succ_b.rolling(self.time_param,self.time_param).mean() ** (-12/2)).rank(axis=1,ascending=False)<=20)*1
        
        mrar_succ_zero_melt = mrar_succ_zero.reset_index().melt(id_vars='日期',var_name='代號',value_name='mrar_succ_zero_'+str(self.time_param))
        mrar_succ_rf_melt = mrar_succ_rf.reset_index().melt(id_vars='日期',var_name='代號',value_name='mrar_succ_rf_'+str(self.time_param))
        mrar_succ_c_melt = mrar_succ_c.reset_index().melt(id_vars='日期',var_name='代號',value_name='mrar_succ_c_'+str(self.time_param))
        mrar_succ_b_melt = mrar_succ_b.reset_index().melt(id_vars='日期',var_name='代號',value_name='mrar_succ_b_'+str(self.time_param))
        df_mrar_succ[mrar_succ_zero_melt.columns] = mrar_succ_zero_melt
        df_mrar_succ[mrar_succ_rf_melt.columns] = mrar_succ_rf_melt
        df_mrar_succ[mrar_succ_c_melt.columns] = mrar_succ_c_melt
        df_mrar_succ[mrar_succ_b_melt.columns] = mrar_succ_b_melt
        return df_mrar_succ
    def hitrate(self):
        df_hitrate   = pd.DataFrame()
        hitrate_zero = (self.fund_return[self.fund_list].subtract(0, axis=0) >0)*1
        hitrate_zero = hitrate_zero.rolling(self.time_param,self.time_param).mean()

        hitrate_rf   = (self.fund_return[self.fund_list].subtract(self.fund_return[self.rf_col].values, axis=0) >0)*1
        hitrate_rf   = hitrate_rf.rolling(self.time_param,self.time_param).mean()

        hitrate_c    = (self.fund_return[self.fund_list].subtract(self.fund_return[self.category_average_col].values, axis=0) >0)*1
        hitrate_c    = hitrate_c.rolling(self.time_param,self.time_param).mean()

        hitrate_b    = (self.fund_return[self.fund_list].subtract(self.fund_return[self.benchmark_col].values, axis=0) >0)*1
        hitrate_b    = hitrate_b.rolling(self.time_param,self.time_param).mean()
        
        hitrate_zero_melt = hitrate_zero.reset_index().melt(id_vars='日期',var_name='代號',value_name='hitrate_zero_'+str(self.time_param))
        hitrate_rf_melt = hitrate_rf.reset_index().melt(id_vars='日期',var_name='代號',value_name='hitrate_rf_'+str(self.time_param))
        hitrate_c_melt = hitrate_c.reset_index().melt(id_vars='日期',var_name='代號',value_name='hitrate_c_'+str(self.time_param))
        hitrate_b_melt = hitrate_b.reset_index().melt(id_vars='日期',var_name='代號',value_name='hitrate_b_'+str(self.time_param))
        df_hitrate[hitrate_zero_melt.columns] = hitrate_zero_melt
        df_hitrate[hitrate_rf_melt.columns] = hitrate_rf_melt
        df_hitrate[hitrate_c_melt.columns] = hitrate_c_melt
        df_hitrate[hitrate_b_melt.columns] = hitrate_b_melt
        return df_hitrate
    def sharpe(self):
        df_sharpe   = pd.DataFrame()
        sharpe_zero = self.fund_return[self.fund_list].subtract(0, axis=0)
        sharpe_zero = sharpe_zero.rolling(self.time_param,self.time_param).mean()/sharpe_zero.rolling(self.time_param,self.time_param).std()

        sharpe_rf   = self.fund_return[self.fund_list].subtract(self.fund_return[self.rf_col].values, axis=0)
        sharpe_rf   = sharpe_rf.rolling(self.time_param,self.time_param).mean()/sharpe_rf.rolling(self.time_param,self.time_param).std()

        sharpe_c    = self.fund_return[self.fund_list].subtract(self.fund_return[self.category_average_col].values, axis=0)
        sharpe_c    = sharpe_c.rolling(self.time_param,self.time_param).mean()/sharpe_c.rolling(self.time_param,self.time_param).std()

        sharpe_b    = self.fund_return[self.fund_list].subtract(self.fund_return[self.benchmark_col].values, axis=0)
        sharpe_b    = sharpe_b.rolling(self.time_param,self.time_param).mean()/sharpe_b.rolling(self.time_param,self.time_param).std()
        
        sharpe_zero_melt = sharpe_zero.reset_index().melt(id_vars='日期',var_name='代號',value_name='sharpe_zero_'+str(self.time_param))
        sharpe_rf_melt = sharpe_rf.reset_index().melt(id_vars='日期',var_name='代號',value_name='sharpe_rf_'+str(self.time_param))
        sharpe_c_melt = sharpe_c.reset_index().melt(id_vars='日期',var_name='代號',value_name='sharpe_c_'+str(self.time_param))
        sharpe_b_melt = sharpe_b.reset_index().melt(id_vars='日期',var_name='代號',value_name='sharpe_b_'+str(self.time_param))
        df_sharpe[sharpe_zero_melt.columns] = sharpe_zero_melt
        df_sharpe[sharpe_rf_melt.columns] = sharpe_rf_melt
        df_sharpe[sharpe_c_melt.columns] = sharpe_c_melt
        df_sharpe[sharpe_b_melt.columns] = sharpe_b_melt
        return df_sharpe

In [ ]:
index_cols = [ 'TTW50T Index', 'IBOXIG Index', 'IBOXHY Index', 'LT11TRUU Index',
       'TWSEDVD Index', 'RMSG Index' ]
rf_col     = ['GVTW10YR Index']

df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
df_index.columns=df_index.columns[1:].insert(0,'date')
df_index = df_index.set_index('date')
df_index.index =[ int(datetime.strftime(x,'%Y%m%d')) for x in df_index.index ]
df_index = df_index.fillna(method='ffill')
df_rf    = df_index[rf_col]
df_rf    = df_rf/100/250 #/100百分比 #/250 日化
df_index = df_index[index_cols]

df_factor_all = pd.DataFrame()
fund_return = pd.DataFrame()
for key in group_fund_dict.keys():
    fund_list = list(set(df_fund_nav.columns)&set(group_fund_dict[key]))
    temp = df_fund_nav[fund_list].copy()
    benchmark_col = key                ##
    category_average_col =key+'_avg'   ##
    temp[benchmark_col]     = df_index[benchmark_col].copy()
    temp[benchmark_col]     = fill_na_until_last_valid(temp[benchmark_col])
    temp[rf_col]            = df_rf[rf_col].copy()
    temp[rf_col]            = fill_na_until_last_valid(temp[rf_col]).fillna(0)            
    fund_return[fund_list]      = temp[fund_list]/temp[fund_list].shift(1)-1
    fund_return[benchmark_col]  = temp[benchmark_col]/temp[benchmark_col].shift(1)-1
    fund_return[rf_col]         = temp[rf_col] 
    fund_return[category_average_col] = fund_return[fund_list].mean(axis=1)
    fund_return.count(axis=1).plot()
    plt.show()
    time_params_list   = [ 20,60,120,240,500,750 ] 
    df_factor = pd.DataFrame()
    for i in range(len(time_params_list)):
        time_param = time_params_list[i]                                                                             ##
        performance_f=performance_factor(fund_return,fund_list,rf_col,category_average_col,benchmark_col,time_param) ##
        df_factor[performance_f.aer().columns]       = performance_f.aer()
        df_factor[performance_f.aer_succ().columns]  = performance_f.aer_succ()
        df_factor[performance_f.mrar().columns]      = performance_f.mrar()
        df_factor[performance_f.mrar_succ().columns] = performance_f.mrar_succ()
        df_factor[performance_f.hitrate().columns]   = performance_f.hitrate()
        df_factor[performance_f.sharpe().columns]    = performance_f.sharpe()
    df_factor_all = pd.concat([df_factor_all ,df_factor])

In [ ]:
    df_factor_all =     df_factor_all.dropna()

# 策略區

In [ ]:
def read_factor(df_factor_all,factor,time_param):
    factor_col = factor+'_'+str(time_param)
    return df_factor_all[['日期','代號',factor_col]]

def fund_select(df_factor,max_n_fund,factor,time_param,df_fund_info):
    factor_col = factor+'_'+str(time_param)
    df_factor_group = pd.merge(left=df_factor,right=df_fund_info[['代號','基金分組']],on=['代號'])
    df_selected_fund = df_factor_group.groupby(['日期','基金分組']).apply(lambda x : x.sort_values(factor_col).tail(max_n_fund)['代號']).reset_index().drop('level_2',axis=1)
    df_selected_fund.columns = ['date','group','ticker']
    df_selected_fund['month'] = [ str(x)[:6] for x in df_selected_fund['date'] ]
    df_selected_fund = df_selected_fund.sort_values(['date','group'])
    #更改換倉頻率 - 日轉月頻
    df_date = df_selected_fund[['date','month']].drop_duplicates(subset=['month'],keep='last')
    #合併月底資料，代表換倉日
    df_selected_fund_date = pd.merge(left=df_selected_fund,right=df_date,on=['date','month'])    
    return df_selected_fund_date

def cal_weight(df_selected_fund_date,asset_weight='opt_weight_sp_1.0'):
    #匯入上層各組權重
    df_group_weight = pd.read_csv('./asset_weight/{}.csv'.format(asset_weight))
    df_group_weight['month'] = df_group_weight['month'].astype('str')
    #df_group_weight.pivot(index='month',columns='index',values='opt_weight_sp_1.0').plot.area(figsize=(15,7.5)).legend(bbox_to_anchor=(1.2, 0.5))
    plt.show()
    df_selected_fund_date_group = pd.merge(left=df_selected_fund_date,right=df_group_weight,left_on=['month','group'],right_on=['month','index'])
    df_selected_fund_date_group['n_securities_per_date_group'] = df_selected_fund_date_group.groupby(['date','group'])['ticker'].transform(lambda x : x.count())
    df_selected_fund_date_group['n_group_per_date'] = df_selected_fund_date_group.groupby(['date'])['group'].transform(lambda x : x.nunique())

    #計算組間權重
    df_selected_fund_date_group['intergroup_weight'] = df_selected_fund_date_group['opt_weight_sp_1.0']
    #計算組內權重
    df_selected_fund_date_group['intragroup_weight'] = 1/df_selected_fund_date_group['n_securities_per_date_group']
    #計算基金權重
    df_selected_fund_date_group['weight'] = df_selected_fund_date_group['intragroup_weight'] *  df_selected_fund_date_group['intergroup_weight'] 
    #df_selected_fund_date_group.groupby('date')['weight'].sum().plot()
    return df_selected_fund_date_group

def cal_return( df_fund_return,df_selected_fund_date_group):
    df_fund_return['date']       = df_fund_return.index
    df_fund_return_melt          = df_fund_return.melt(id_vars='date',var_name='ticker',value_name='return')
    df_fund_return_melt          = df_fund_return_melt.dropna()
    df_fund_return_melt['month'] = df_fund_return_melt['date'].apply(make_rebal_key)

    df_fund_return_melt.ticker   = df_fund_return_melt.ticker.astype('str')
    df_fund_return_melt.month    = df_fund_return_melt.month.astype('str')
    df_selected_fund_date_group.ticker = df_selected_fund_date_group.ticker.astype('str')
    df_selected_fund_date_group.month = df_selected_fund_date_group.month.astype('str')

    df_final           = pd.merge(left = df_fund_return_melt,right =df_selected_fund_date_group[['ticker','month','group','weight']],on=['ticker','month'] )
    df_final['wxr']    = df_final['weight']*df_final['return']
    df_final['date'] = pd.to_datetime(df_final['date'],format='%Y%m%d')
    df_final           = df_final.sort_values(['date','month','group'])
    return df_final 

def performance_stats(Return,Return_Benchmark):
    #權益或是累積報酬
    Return = pd.Series(Return)
    Equity = (Return+1).cumprod()
    Geo_a_Return_annual   = (Equity[-1])**(250/len(Equity))-1
    Arith_a_Return_annual = (Return.mean()+1)**(250)-1
    #投組勝率
    Hitrate =sum(Return>0)/len(Return)
    Return_p_values     = stats.ttest_1samp(Return, 0).pvalue
    STD_Return_annual   = Return.std()*np.sqrt(250)

    Sharpe_annual = (Geo_a_Return_annual) / STD_Return_annual

    #計算MDD
    D = Equity.cummax() - Equity
    MDD = D.max()*1
    d = D / (D +Equity)
    mdd = d.max()

    #投組勝率
    winrate =sum((Return.values-Return_Benchmark.values.flatten())>0)/len(Return)
    #贏過大盤pvalues
    win_Return_p_values     = stats.ttest_1samp(Return.values-Return_Benchmark.values.flatten(), 0).pvalue
    
    result = pd.Series([Equity[-1],Geo_a_Return_annual,Arith_a_Return_annual,
                        STD_Return_annual,Sharpe_annual,mdd,
                        Return_p_values,Hitrate,
                        win_Return_p_values ,winrate
                       ])
    result.index = ['Total Return','Geometric Mean Return','Arithmetric Mean Return',
                    'Standard Deviation','Sharpe Ratios','mdd',
                    'P-Value','Hit Rate',
                    'Win-P-Value','Win Rate']
    return  result.round(2)

def plot_performance_save(df_final,file_name,strategy_name ):
    df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
    df_index.columns=df_index.columns[1:].insert(0,'date')
    df_index = df_index.set_index('date')
    df_index = df_index.fillna(method='ffill')
    df_index = df_index.dropna()
    df_index = df_index.loc[df_index.index    >=datetime.strptime('20050101','%Y%m%d')]
    df_final = df_final.loc[df_final['date']>=df_index.index.min()]
    df_index = df_index[index_cols]

    df_return = pd.DataFrame()
    df_return[strategy_name]    = df_final.groupby('date')['wxr'].sum()
    df_return[df_index.columns] = df_index
    df_return[df_index.columns] = df_return[df_index.columns].fillna(method='ffill').pct_change().fillna(0)
    (df_return+1).cumprod().plot(figsize=(15,7.5))
    plt.grid()
    plt.savefig('F:/DataScienceTeam/基金篩選-量化部門/result/{}/result_{}.png'.format(file_name,strategy_name))
    plt.show()
    df_final.groupby('date')['weight'].sum().plot()
    plt.show()
    benchmark_cols  = ['TWSEDVD Index']

    df_return.apply(lambda x : performance_stats(x,df_return[benchmark_cols])).to_csv('F:/DataScienceTeam/基金篩選-量化部門/result/{}/result_{}.csv'.format(file_name,strategy_name))
    df_return.apply(lambda x : performance_stats(x,df_return[benchmark_cols]))    

def strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info):
    for i in range(len(time_params_list)):
        for j in range(len(max_n_fund_list)):
            time_param = time_params_list[i]
            max_n_fund = max_n_fund_list[j]
            file_name = 'inter_{}_intra_{}_{}'.format(inter_strategy,other,factor )
            if not os.path.exists('F:/DataScienceTeam/基金篩選-量化部門/result/'+file_name):
                os.mkdir('F:/DataScienceTeam/基金篩選-量化部門/result/'+file_name)
            strategy_name = file_name+'_{}_{}'.format(time_param,max_n_fund )
            print('strategy name :{}'.format(strategy_name))    
            ###########################################################################################################
            #factor
            ########################################################################################################### 
            df_factor                   = read_factor(df_factor_all,factor,time_param)
            ###########################################################################################################
            #篩選基金
            ########################################################################################################### 
            df_selected_fund_date       = fund_select(df_factor,max_n_fund,factor,time_param,df_fund_info)
            ###########################################################################################################
            #計算權重
            ###########################################################################################################         
            df_selected_fund_date_group = cal_weight(df_selected_fund_date,asset_weight='opt_weight_sp_1.0')
            ###########################################################################################################
            #合併日報酬資料與月換倉資料、計算報酬
            ###########################################################################################################  
            df_final                    = cal_return( df_fund_return,df_selected_fund_date_group)
            ###########################################################################################################
            #作圖比較指數及策略績效
            ########################################################################################################### 
            plot_performance_save(df_final,file_name,strategy_name )

# 一、組間極大化夏普上限1 - 組內d日aer最大n檔

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'aer_zero'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'aer_rf'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'aer_c'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'aer_b'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

# 二、組間極大化夏普上限1 - 組內d日mrar最大n檔

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'mrar_zero'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'mrar_rf'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'mrar_c'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'mrar_b'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

# 三、組間極大化夏普上限1 - 組內d日sharpe最大n檔

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'sharpe_zero'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_rf'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_c'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_b'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

# 四、組間極大化夏普上限1 - 組內d日hitrate最大n檔

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'hitrate_zero'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_rf'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_c'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_b'
other  = ''
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info)

# 五、組間極大化夏普上限1 - 組內d日aer最大n檔，債券換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'aer_zero'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_rf'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_c'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_b'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 六、組間極大化夏普上限1 - 組內d日mrar最大n檔，債券換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'mrar_zero'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_rf'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_c'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_b'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 七、組間極大化夏普上限1 - 組內d日sharpe最大n檔，債券換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'sharpe_zero'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_rf'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_c'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_b'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 八、組間極大化夏普上限1 - 組內d日hitrate最大n檔，債券換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'hitrate_zero'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_rf'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_c'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_b'
other  = 'bondtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 九、組間極大化夏普上限1 - 組內d日aer最大n檔，IG換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'aer_zero'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_rf'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_c'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_b'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 十、組間極大化夏普上限1 - 組內d日mrar最大n檔，IG換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'mrar_zero'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_rf'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_c'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_b'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 十一、組間極大化夏普上限1 - 組內d日sharpe最大n檔，IG換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'sharpe_zero'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_rf'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_c'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_b'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 十二、組間極大化夏普上限1 - 組內d日hitrate最大n檔，IG換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'hitrate_zero'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_rf'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_c'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_b'
other  = 'igtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 十三、組間極大化夏普上限1 - 組內d日aer最大n檔，TB換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'aer_zero'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_rf'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_c'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'aer_b'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 十四、組間極大化夏普上限1 - 組內d日mrar最大n檔，TB換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'mrar_zero'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_rf'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_c'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'mrar_b'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 十五、組間極大化夏普上限1 - 組內d日sharpe最大n檔，TB換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'sharpe_zero'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_rf'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_c'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'sharpe_b'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

# 十六、組間極大化夏普上限1 - 組內d日hitrate最大n檔，TB換指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']
df_etf_info           = pd.DataFrame(
                         [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                          ['','IBOXIG Index','','','','','','IBOXIG Index']
                         ],columns=df_fund_info.columns)##與上一個策略不同
df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'hitrate_zero'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_rf'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_c'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

inter_strategy = 'maxsharpe1'
factor = 'hitrate_b'
other  = 'tbtoindex'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
strategy(inter_strategy,df_factor_all,factor,time_params_list,max_n_fund_list,other,df_fund_info_etf)

In [ ]:
###############################################################################################################################

# 以前的code ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# 二、極大化夏普上限1 - 組內d日MRAR最大n檔，債券換成指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'bondtoindex_mrar'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 6,7,8,9,10 ]
for i in range(len(time_params_list)):
    for j in range(len(max_n_fund_list)):
        time_param = time_params_list[i]
        max_n_fund = max_n_fund_list[j]
        file_name = 'inter_{}_intra_{}'.format(inter_strategy,factor )
        if not os.path.exists('F:/DataScienceTeam/基金篩選-量化部門/result/'+file_name):
            os.mkdir('F:/DataScienceTeam/基金篩選-量化部門/result/'+file_name)
        strategy_name = file_name+'_{}_{}'.format(time_param,max_n_fund )
        print('strategy name :{}'.format(strategy_name))
        ###########################################################################################################
        #讀取指數data
        ###########################################################################################################
        df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
        df_index.columns=df_index.columns[1:].insert(0,'date')
        df_index = df_index.set_index('date')
        df_index.index =[ int(datetime.strftime(x,'%Y%m%d')) for x in df_index.index ]
        df_index = df_index.fillna(method='ffill')
        df_rf    = df_index[rf_col]
        df_rf    = df_rf/100/250 #/100百分比 #/250 日化
        df_index = df_index[index_cols]
        ###########################################################################################################
        #計算factor
        ###########################################################################################################
        df_fund_mrar          = pd.DataFrame()
        for key in group_fund_dict.keys():

            fund_list = list(set(df_fund_nav.columns)&set(group_fund_dict[key]))
            temp = df_fund_nav[fund_list].copy()
            benchmark_col = key
            temp[benchmark_col]     = df_index[benchmark_col].copy()
            temp[benchmark_col]     = fill_na_until_last_valid(temp[benchmark_col])
            temp[rf_col]            = df_rf[rf_col].copy()
            temp[rf_col]            = fill_na_until_last_valid(temp[rf_col]).fillna(0)            
            fund_return   = temp/temp.shift(1)

            if fund_list[0] == key:
                fund_excess_return = fund_return[fund_list]
            else:
                fund_excess_return = fund_return[fund_list].apply(lambda x : x/fund_return[benchmark_col])

            df_fund_mrar[fund_list]          = (fund_excess_return )** -2
            df_fund_mrar[fund_list]          = (df_fund_mrar[fund_list].rolling(time_param,time_param).mean() ** -time_param/2) -1 

        df_fund_mrar['date'] = df_fund_mrar.index
        df_mrar              = df_fund_mrar.melt(id_vars='date',value_name='mrar',var_name='代號').dropna()
        df_mrar              = df_mrar.sort_values(['date','代號'])

        df_etf_info           = pd.DataFrame(
                                 [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                                  ['','IBOXIG Index','','','','','','IBOXIG Index']
                                 ],columns=df_fund_info.columns)
        df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])
        df_mrar_group = pd.merge(left=df_mrar,right=df_fund_info_etf[['代號','基金分組']],on=['代號'])
        ###########################################################################################################
        #篩選基金
        ###########################################################################################################        
        df_etf_info           = pd.DataFrame(
                                 [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                                  ['','IBOXIG Index','','','','','','IBOXIG Index']
                                 ],columns=df_fund_info.columns)##與上一個策略不同
        df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同
        df_mrar_group = pd.merge(left=df_mrar,right=df_fund_info_etf[['代號','基金分組']],on=['代號'])
        df_selected_fund = df_mrar_group.groupby(['date','基金分組']).apply(lambda x : x.sort_values('mrar').tail(max_n_fund)['代號']).reset_index().drop('level_2',axis=1)
        df_selected_fund.columns = ['date','group','ticker']
        df_selected_fund['month'] = [ str(x)[:6] for x in df_selected_fund['date'] ]
        df_selected_fund = df_selected_fund.sort_values(['date','group'])
        ###########################################################################################################
        #更改換倉頻率 - 日轉月頻
        ###########################################################################################################          
        df_date = df_selected_fund[['date','month']].drop_duplicates(subset=['month'],keep='last')
        #合併月底資料，代表換倉日
        df_selected_fund_date = pd.merge(left=df_selected_fund,right=df_date,on=['date','month'])
        ###########################################################################################################
        #計算權重
        ###########################################################################################################         
        #匯入上層各組權重
        df_group_weight = pd.read_csv('./asset_weight/opt_weight_sp_1.0.csv')
        df_group_weight['month'] = df_group_weight['month'].astype('str')
        #df_group_weight.pivot(index='month',columns='index',values='opt_weight_sp_1.0').plot.area(figsize=(15,7.5)).legend(bbox_to_anchor=(1.2, 0.5))
        plt.show()
        df_selected_fund_date_group = pd.merge(left=df_selected_fund_date,right=df_group_weight,left_on=['month','group'],right_on=['month','index'])
        df_selected_fund_date_group['n_securities_per_date_group'] = df_selected_fund_date_group.groupby(['date','group'])['ticker'].transform(lambda x : x.count())
        df_selected_fund_date_group['n_group_per_date'] = df_selected_fund_date_group.groupby(['date'])['group'].transform(lambda x : x.nunique())

        #計算組間權重
        df_selected_fund_date_group['intergroup_weight'] = df_selected_fund_date_group['opt_weight_sp_1.0']
        #計算組內權重
        df_selected_fund_date_group['intragroup_weight'] = 1/df_selected_fund_date_group['n_securities_per_date_group']
        #計算基金權重
        df_selected_fund_date_group['weight'] = df_selected_fund_date_group['intragroup_weight'] *  df_selected_fund_date_group['intergroup_weight'] 
        #df_selected_fund_date_group.groupby('date')['weight'].sum().plot()
        ###########################################################################################################
        #合併日報酬資料與月換倉資料、計算報酬
        ###########################################################################################################           
        df_fund_return['date'] = df_fund_return.index
        df_fund_return_melt = df_fund_return.melt(id_vars='date',var_name='ticker',value_name='return')
        df_fund_return_melt = df_fund_return_melt.dropna()
        df_fund_return_melt['month'] = df_fund_return_melt['date'].apply(make_rebal_key)

        df_fund_return_melt.ticker   = df_fund_return_melt.ticker.astype('str')
        df_fund_return_melt.month    = df_fund_return_melt.month.astype('str')
        df_selected_fund_date_group.ticker = df_selected_fund_date_group.ticker.astype('str')
        df_selected_fund_date_group.month = df_selected_fund_date_group.month.astype('str')
        df_final           = pd.merge(left = df_fund_return_melt,right =df_selected_fund_date_group,on=['ticker','month'] )
        df_final['wxr']    = df_final['weight']*df_final['return']
        df_final['date_x'] = pd.to_datetime(df_final['date_x'],format='%Y%m%d')
        df_final           = df_final.sort_values(['date_x','month','group'])
        ###########################################################################################################
        #作圖比較指數及策略績效
        ########################################################################################################### 
        df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
        df_index.columns=df_index.columns[1:].insert(0,'date')
        df_index = df_index.set_index('date')
        df_index = df_index.fillna(method='ffill')
        df_index = df_index.dropna()
        df_index = df_index.loc[df_index.index    >=datetime.strptime('20050101','%Y%m%d')]
        df_index = df_index[index_cols]
        df_final = df_final.loc[df_final['date_x']>=df_index.index.min()]

        df_return = pd.DataFrame()
        df_return[strategy_name]    = df_final.groupby('date_x')['wxr'].sum()
        df_return[df_index.columns] = df_index
        df_return[df_index.columns] = df_return[df_index.columns].fillna(method='ffill').pct_change().fillna(0)
        (df_return+1).cumprod().plot(figsize=(15,7.5))
        plt.grid()
        plt.savefig('F:/DataScienceTeam/基金篩選-量化部門/result/{}/result_{}.png'.format(file_name,strategy_name))
        plt.show()
        df_final.groupby('date_x')['weight'].sum().plot()
        plt.show()
        benchmark_cols  = ['TWSEDVD Index']

        df_return.apply(lambda x : performance_stats(x,df_return[benchmark_cols])).to_csv('F:/DataScienceTeam/基金篩選-量化部門/result/{}/result_{}.csv'.format(file_name,strategy_name))
        df_return.apply(lambda x : performance_stats(x,df_return[benchmark_cols]))

# 三、MRAR最大一檔 - 指數作為benchmark，IG換成指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['IBOXIG Index'] = ['IBOXIG Index']

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'IGtoindex_mrar'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5,6,7,8,9,10 ]
for i in range(len(time_params_list)):
    for j in range(len(max_n_fund_list)):
        time_param = time_params_list[i]
        max_n_fund = max_n_fund_list[j]
        file_name = 'inter_{}_intra_{}'.format(inter_strategy,factor )
        if not os.path.exists('F:/DataScienceTeam/基金篩選-量化部門/result/'+file_name):
            os.mkdir('F:/DataScienceTeam/基金篩選-量化部門/result/'+file_name)
        strategy_name = file_name+'_{}_{}'.format(time_param,max_n_fund )
        print('strategy name :{}'.format(strategy_name))
        ###########################################################################################################
        #讀取指數data
        ###########################################################################################################
        df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
        df_index.columns=df_index.columns[1:].insert(0,'date')
        df_index = df_index.set_index('date')
        df_index.index =[ int(datetime.strftime(x,'%Y%m%d')) for x in df_index.index ]
        df_index = df_index.fillna(method='ffill')
        df_rf    = df_index[rf_col]
        df_rf    = df_rf/100/250 #/100百分比 #/250 日化
        df_index = df_index[index_cols]
        ###########################################################################################################
        #計算factor
        ###########################################################################################################
        df_fund_mrar          = pd.DataFrame()
        for key in group_fund_dict.keys():

            fund_list = list(set(df_fund_nav.columns)&set(group_fund_dict[key]))
            temp = df_fund_nav[fund_list].copy()
            benchmark_col = key
            temp[benchmark_col]     = df_index[benchmark_col].copy()
            temp[benchmark_col]     = fill_na_until_last_valid(temp[benchmark_col])
            temp[rf_col]            = df_rf[rf_col].copy()
            temp[rf_col]            = fill_na_until_last_valid(temp[rf_col]).fillna(0)            
            fund_return   = temp/temp.shift(1)

            if fund_list[0] == key:
                fund_excess_return = fund_return[fund_list]
            else:
                fund_excess_return = fund_return[fund_list].apply(lambda x : x/fund_return[benchmark_col])

            df_fund_mrar[fund_list]          = (fund_excess_return )** -2
            df_fund_mrar[fund_list]          = (df_fund_mrar[fund_list].rolling(time_param,time_param).mean() ** -time_param/2) -1 

        df_fund_mrar['date'] = df_fund_mrar.index
        df_mrar              = df_fund_mrar.melt(id_vars='date',value_name='mrar',var_name='代號').dropna()
        df_mrar              = df_mrar.sort_values(['date','代號'])

        df_etf_info           = pd.DataFrame(
                                 [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                                  ['','IBOXIG Index','','','','','','IBOXIG Index']
                                 ],columns=df_fund_info.columns)
        df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])
        df_mrar_group = pd.merge(left=df_mrar,right=df_fund_info_etf[['代號','基金分組']],on=['代號'])
        ###########################################################################################################
        #篩選基金
        ###########################################################################################################        
        df_etf_info           = pd.DataFrame(
                                 [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                                  ['','IBOXIG Index','','','','','','IBOXIG Index']
                                 ],columns=df_fund_info.columns)##與上一個策略不同
        df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同
        df_mrar_group = pd.merge(left=df_mrar,right=df_fund_info_etf[['代號','基金分組']],on=['代號'])
        df_selected_fund = df_mrar_group.groupby(['date','基金分組']).apply(lambda x : x.sort_values('mrar').tail(max_n_fund)['代號']).reset_index().drop('level_2',axis=1)
        df_selected_fund.columns = ['date','group','ticker']
        df_selected_fund['month'] = [ str(x)[:6] for x in df_selected_fund['date'] ]
        df_selected_fund = df_selected_fund.sort_values(['date','group'])
        ###########################################################################################################
        #更改換倉頻率 - 日轉月頻
        ###########################################################################################################          
        df_date = df_selected_fund[['date','month']].drop_duplicates(subset=['month'],keep='last')
        #合併月底資料，代表換倉日
        df_selected_fund_date = pd.merge(left=df_selected_fund,right=df_date,on=['date','month'])
        ###########################################################################################################
        #計算權重
        ###########################################################################################################         
        #匯入上層各組權重
        df_group_weight = pd.read_csv('./asset_weight/opt_weight_sp_1.0.csv')
        df_group_weight['month'] = df_group_weight['month'].astype('str')
        #df_group_weight.pivot(index='month',columns='index',values='opt_weight_sp_1.0').plot.area(figsize=(15,7.5)).legend(bbox_to_anchor=(1.2, 0.5))
        plt.show()
        df_selected_fund_date_group = pd.merge(left=df_selected_fund_date,right=df_group_weight,left_on=['month','group'],right_on=['month','index'])
        df_selected_fund_date_group['n_securities_per_date_group'] = df_selected_fund_date_group.groupby(['date','group'])['ticker'].transform(lambda x : x.count())
        df_selected_fund_date_group['n_group_per_date'] = df_selected_fund_date_group.groupby(['date'])['group'].transform(lambda x : x.nunique())

        #計算組間權重
        df_selected_fund_date_group['intergroup_weight'] = df_selected_fund_date_group['opt_weight_sp_1.0']
        #計算組內權重
        df_selected_fund_date_group['intragroup_weight'] = 1/df_selected_fund_date_group['n_securities_per_date_group']
        #計算基金權重
        df_selected_fund_date_group['weight'] = df_selected_fund_date_group['intragroup_weight'] *  df_selected_fund_date_group['intergroup_weight'] 
        #df_selected_fund_date_group.groupby('date')['weight'].sum().plot()
        ###########################################################################################################
        #合併日報酬資料與月換倉資料、計算報酬
        ###########################################################################################################           
        df_fund_return['date'] = df_fund_return.index
        df_fund_return_melt = df_fund_return.melt(id_vars='date',var_name='ticker',value_name='return')
        df_fund_return_melt = df_fund_return_melt.dropna()
        df_fund_return_melt['month'] = df_fund_return_melt['date'].apply(make_rebal_key)

        df_fund_return_melt.ticker   = df_fund_return_melt.ticker.astype('str')
        df_fund_return_melt.month    = df_fund_return_melt.month.astype('str')
        df_selected_fund_date_group.ticker = df_selected_fund_date_group.ticker.astype('str')
        df_selected_fund_date_group.month = df_selected_fund_date_group.month.astype('str')
        df_final           = pd.merge(left = df_fund_return_melt,right =df_selected_fund_date_group,on=['ticker','month'] )
        df_final['wxr']    = df_final['weight']*df_final['return']
        df_final['date_x'] = pd.to_datetime(df_final['date_x'],format='%Y%m%d')
        df_final           = df_final.sort_values(['date_x','month','group'])
        ###########################################################################################################
        #作圖比較指數及策略績效
        ########################################################################################################### 
        df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
        df_index.columns=df_index.columns[1:].insert(0,'date')
        df_index = df_index.set_index('date')
        df_index = df_index.fillna(method='ffill')
        df_index = df_index.dropna()
        df_index = df_index.loc[df_index.index    >=datetime.strptime('20050101','%Y%m%d')]
        df_index = df_index[index_cols]
        df_final = df_final.loc[df_final['date_x']>=df_index.index.min()]

        df_return = pd.DataFrame()
        df_return[strategy_name]    = df_final.groupby('date_x')['wxr'].sum()
        df_return[df_index.columns] = df_index
        df_return[df_index.columns] = df_return[df_index.columns].fillna(method='ffill').pct_change().fillna(0)
        (df_return+1).cumprod().plot(figsize=(15,7.5))
        plt.grid()
        plt.savefig('F:/DataScienceTeam/基金篩選-量化部門/result/{}/result_{}.png'.format(file_name,strategy_name))
        plt.show()
        df_final.groupby('date_x')['weight'].sum().plot()
        plt.show()
        benchmark_cols  = ['TWSEDVD Index']

        df_return.apply(lambda x : performance_stats(x,df_return[benchmark_cols])).to_csv('F:/DataScienceTeam/基金篩選-量化部門/result/{}/result_{}.csv'.format(file_name,strategy_name))
        df_return.apply(lambda x : performance_stats(x,df_return[benchmark_cols]))

# 四、極大化夏普上限1 - 組內d日MRAR最大n檔，公債換成指數

In [ ]:
group_fund_dict = {}
for key in group_dict.keys():
    group_fund_dict[key] = list(df_fund_info[df_fund_info['基金分組']==key]['代號'])
group_fund_dict['LT11TRUU Index'] = ['LT11TRUU Index']

In [ ]:
inter_strategy = 'maxsharpe1'
factor = 'TBtoindex_mrar'
time_params_list   = [ 20,60,120,240,500,750 ]
max_n_fund_list    = [ 1,2,3,4,5,6,7,8,9,10 ]
for i in range(len(time_params_list)):
    for j in range(len(max_n_fund_list)):
        time_param = time_params_list[i]
        max_n_fund = max_n_fund_list[j]
        file_name = 'inter_{}_intra_{}'.format(inter_strategy,factor )
        if not os.path.exists('F:/DataScienceTeam/基金篩選-量化部門/result/'+file_name):
            os.mkdir('F:/DataScienceTeam/基金篩選-量化部門/result/'+file_name)
        strategy_name = file_name+'_{}_{}'.format(time_param,max_n_fund )
        print('strategy name :{}'.format(strategy_name))
        ###########################################################################################################
        #讀取指數data
        ###########################################################################################################
        df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
        df_index.columns=df_index.columns[1:].insert(0,'date')
        df_index = df_index.set_index('date')
        df_index.index =[ int(datetime.strftime(x,'%Y%m%d')) for x in df_index.index ]
        df_index = df_index.fillna(method='ffill')
        df_rf    = df_index[rf_col]
        df_rf    = df_rf/100/250 #/100百分比 #/250 日化
        df_index = df_index[index_cols]
        ###########################################################################################################
        #計算factor
        ###########################################################################################################  
        
        df_fund_mrar          = pd.DataFrame()
        for key in group_fund_dict.keys():

            fund_list = list(set(df_fund_nav.columns)&set(group_fund_dict[key]))
            temp = df_fund_nav[fund_list].copy()
            benchmark_col = key
            temp[benchmark_col]     = df_index[benchmark_col].copy()
            temp[benchmark_col]     = fill_na_until_last_valid(temp[benchmark_col])
            temp[rf_col]            = df_rf[rf_col].copy()
            temp[rf_col]            = fill_na_until_last_valid(temp[rf_col]).fillna(0)            
            fund_return   = temp/temp.shift(1)

            if fund_list[0] == key:
                fund_excess_return = fund_return[fund_list]
            else:
                fund_excess_return = fund_return[fund_list].apply(lambda x : x/fund_return[benchmark_col])

            df_fund_mrar[fund_list]          = (fund_excess_return )** -2
            df_fund_mrar[fund_list]          = (df_fund_mrar[fund_list].rolling(time_param,time_param).mean() ** -time_param/2) -1 

        df_fund_mrar['date'] = df_fund_mrar.index
        df_mrar              = df_fund_mrar.melt(id_vars='date',value_name='mrar',var_name='代號').dropna()
        df_mrar              = df_mrar.sort_values(['date','代號'])

        df_etf_info           = pd.DataFrame(
                                 [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                                  ['','IBOXIG Index','','','','','','IBOXIG Index']
                                 ],columns=df_fund_info.columns)
        df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])
        df_mrar_group = pd.merge(left=df_mrar,right=df_fund_info_etf[['代號','基金分組']],on=['代號'])
        ###########################################################################################################
        #篩選基金
        ###########################################################################################################        
        df_etf_info           = pd.DataFrame(
                                 [['','LT11TRUU Index','','','','','','LT11TRUU Index'],
                                  ['','IBOXIG Index','','','','','','IBOXIG Index']
                                 ],columns=df_fund_info.columns)##與上一個策略不同
        df_fund_info_etf = pd.concat([df_fund_info,df_etf_info])##與上一個策略不同
        df_mrar_group = pd.merge(left=df_mrar,right=df_fund_info_etf[['代號','基金分組']],on=['代號'])
        df_selected_fund = df_mrar_group.groupby(['date','基金分組']).apply(lambda x : x.sort_values('mrar').tail(max_n_fund)['代號']).reset_index().drop('level_2',axis=1)
        df_selected_fund.columns = ['date','group','ticker']
        df_selected_fund['month'] = [ str(x)[:6] for x in df_selected_fund['date'] ]
        df_selected_fund = df_selected_fund.sort_values(['date','group'])
        ###########################################################################################################
        #更改換倉頻率 - 日轉月頻
        ###########################################################################################################          
        df_date = df_selected_fund[['date','month']].drop_duplicates(subset=['month'],keep='last')
        #合併月底資料，代表換倉日
        df_selected_fund_date = pd.merge(left=df_selected_fund,right=df_date,on=['date','month'])
        ###########################################################################################################
        #計算權重
        ###########################################################################################################         
        #匯入上層各組權重
        df_group_weight = pd.read_csv('./asset_weight/opt_weight_sp_1.0.csv')
        df_group_weight['month'] = df_group_weight['month'].astype('str')
        #df_group_weight.pivot(index='month',columns='index',values='opt_weight_sp_1.0').plot.area(figsize=(15,7.5)).legend(bbox_to_anchor=(1.2, 0.5))
        plt.show()
        df_selected_fund_date_group = pd.merge(left=df_selected_fund_date,right=df_group_weight,left_on=['month','group'],right_on=['month','index'])
        df_selected_fund_date_group['n_securities_per_date_group'] = df_selected_fund_date_group.groupby(['date','group'])['ticker'].transform(lambda x : x.count())
        df_selected_fund_date_group['n_group_per_date'] = df_selected_fund_date_group.groupby(['date'])['group'].transform(lambda x : x.nunique())

        #計算組間權重
        df_selected_fund_date_group['intergroup_weight'] = df_selected_fund_date_group['opt_weight_sp_1.0']
        #計算組內權重
        df_selected_fund_date_group['intragroup_weight'] = 1/df_selected_fund_date_group['n_securities_per_date_group']
        #計算基金權重
        df_selected_fund_date_group['weight'] = df_selected_fund_date_group['intragroup_weight'] *  df_selected_fund_date_group['intergroup_weight'] 
        #df_selected_fund_date_group.groupby('date')['weight'].sum().plot()
        ###########################################################################################################
        #合併日報酬資料與月換倉資料、計算報酬
        ###########################################################################################################           
        df_fund_return['date'] = df_fund_return.index
        df_fund_return_melt = df_fund_return.melt(id_vars='date',var_name='ticker',value_name='return')
        df_fund_return_melt = df_fund_return_melt.dropna()
        df_fund_return_melt['month'] = df_fund_return_melt['date'].apply(make_rebal_key)

        df_fund_return_melt.ticker   = df_fund_return_melt.ticker.astype('str')
        df_fund_return_melt.month    = df_fund_return_melt.month.astype('str')
        df_selected_fund_date_group.ticker = df_selected_fund_date_group.ticker.astype('str')
        df_selected_fund_date_group.month = df_selected_fund_date_group.month.astype('str')
        df_final           = pd.merge(left = df_fund_return_melt,right =df_selected_fund_date_group,on=['ticker','month'] )
        df_final['wxr']    = df_final['weight']*df_final['return']
        df_final['date_x'] = pd.to_datetime(df_final['date_x'],format='%Y%m%d')
        df_final           = df_final.sort_values(['date_x','month','group'])
        ###########################################################################################################
        #作圖比較指數及策略績效
        ########################################################################################################### 
        df_index=pd.read_excel('F:/TEMP/Bloomberg暫存區/Data_Team/etf_allocation.xlsx').drop(0)
        df_index.columns=df_index.columns[1:].insert(0,'date')
        df_index = df_index.set_index('date')
        df_index = df_index.fillna(method='ffill')
        df_index = df_index.dropna()
        df_index = df_index.loc[df_index.index    >=datetime.strptime('20050101','%Y%m%d')]
        df_index = df_index[index_cols]
        df_final = df_final.loc[df_final['date_x']>=df_index.index.min()]

        df_return = pd.DataFrame()
        df_return[strategy_name]    = df_final.groupby('date_x')['wxr'].sum()
        df_return[df_index.columns] = df_index
        df_return[df_index.columns] = df_return[df_index.columns].fillna(method='ffill').pct_change().fillna(0)
        (df_return+1).cumprod().plot(figsize=(15,7.5))
        plt.grid()
        plt.savefig('F:/DataScienceTeam/基金篩選-量化部門/result/{}/result_{}.png'.format(file_name,strategy_name))
        plt.show()
        df_final.groupby('date_x')['weight'].sum().plot()
        plt.show()
        benchmark_cols  = ['TWSEDVD Index']

        df_return.apply(lambda x : performance_stats(x,df_return[benchmark_cols])).to_csv('F:/DataScienceTeam/基金篩選-量化部門/result/{}/result_{}.csv'.format(file_name,strategy_name))
        df_return.apply(lambda x : performance_stats(x,df_return[benchmark_cols]))